#6.8 Stuff LCEL Chain

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

In [2]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

@traceable
def generate_response(chain, prompt):
    return chain.invoke(prompt)
